In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
import re

def is_question(text):
    text = str(text).strip()
    if not text or text == 'nan':
        return False
    patterns = [
        r"\?$", 
        r"^(what|how|is|are|do|does|can|who|when|where|why|shall|will|has|have)",
        r"\b(explain|describe|tell me about)\b"
    ]
    return any(re.search(pattern, text.lower()) for pattern in patterns)

def process_row(row):
    for idx, cell in enumerate(row):
        cell_str = str(cell).strip()
        if cell_str.lower() == 'main':
            continue
        if is_question(cell_str):
            return cell_str, list(row[idx+1:])
    return None, []

def format_answer(answer_data):
    formatted = []
    i = 0
    while i < len(answer_data):
        # Clean current row
        row = [
            f"{float(cell)*100:.2f}%" if (isinstance(cell, float) and cell <= 1) else str(cell).strip()
            for cell in answer_data[i]
            if not pd.isnull(cell) and str(cell).strip() not in ['', 'nan']
        ]
        if not row:
            i += 1
            continue

        # Detect table headers with at least 2 columns
        if i + 1 < len(answer_data) and len(row) >= 2:
            # Clean next row
            next_row = [
                f"{float(cell)*100:.2f}%" if (isinstance(cell, float) and cell <= 1) else str(cell).strip()
                for cell in answer_data[i+1]
                if not pd.isnull(cell) and str(cell).strip() not in ['', 'nan']
            ]
            
            if len(next_row) == len(row):
                # Build markdown table
                headers = row
                rows = [next_row]
                i += 2
                
                # Add subsequent matching rows
                while i < len(answer_data):
                    current_row = [
                        f"{float(cell)*100:.2f}%" if (isinstance(cell, float) and cell <= 1) else str(cell).strip()
                        for cell in answer_data[i]
                        if not pd.isnull(cell) and str(cell).strip() not in ['', 'nan']
                    ]
                    if len(current_row) == len(headers):
                        rows.append(current_row)
                        i += 1
                    else:
                        break
                
                # Format table
                table = [
                    f"| {' | '.join(headers)} |",
                    f"| {' | '.join(['---']*len(headers))} |"
                ]
                table.extend([f"| {' | '.join(row)} |" for row in rows])
                formatted.append('\n'.join(table))
                continue

        # Format as list items if not a table
        formatted.extend(f"- {item}" for item in row)
        i += 1
    
    return '\n'.join(formatted)

def process_sheet(sheet_name, df, source):
    markdown = []
    current_q = None
    current_a = []

    for _, row in df.iterrows():
        q, a = process_row(row)
        if q:
            if current_q:
                answer = format_answer(current_a)
                markdown.append(create_block(sheet_name, current_q, answer, source))
            current_q = q
            current_a = [a]
        else:
            cleaned = [cell for cell in row if not pd.isnull(cell)]
            if cleaned:
                current_a.append(cleaned)
    
    if current_q:
        answer = format_answer(current_a)
        markdown.append(create_block(sheet_name, current_q, answer, source))
    
    return '\n'.join(markdown)

def create_block(sheet, q, a, src):
    return f"""---
sheet_name: "{sheet}"
question: "{q}"
source: "{src}"
---

**Answer:**  
{a}

---
"""

excel_file = "/home/zainab/Documents/NUST/Semester 8/Large Language Models sem 8/Project/Code/BankAssist-LLM/data/NUST Bank-Product-Knowledge.xlsx"
output_file = "bank_qna_md.md"

all_sheets = pd.read_excel(excel_file, sheet_name=None, header=None)

with open(output_file, "w", encoding="utf-8") as f:
    for sheet_name, df in all_sheets.items():
        if sheet_name in ["Sheet3", "Sheet1", "Main"]:
            continue
        
        content = process_sheet(sheet_name, df, excel_file)
        if content:
            f.write(content + "\n")

print(f"Markdown output saved to {output_file}")


In [ ]:
import json

def json_to_markdown(json_data, output_file, source_path):
    with open(output_file, "a", encoding="utf-8") as f:
        for category_data in json_data.get("categories", []):
            category = category_data.get("category", "N/A")
            questions = category_data.get("questions", [])

            for qa in questions:
                question = qa.get("question", "").strip()
                answer = qa.get("answer", "").strip()

                f.write('---\n')
                f.write(f'question: "{question}"\n')
                f.write(f'source: "{source_path}"\n')
                f.write('---\n\n')
                f.write('**Answer:**  \n')
                
                for line in answer.split("\n"):
                    if line.strip() == "":
                        f.write("\n")
                    else:
                        f.write(f"- {line.strip()}\n")

                f.write('\n---\n\n')

    print(f"Markdown file written to {output_file}")

input_json_path = "/home/zainab/Documents/NUST/Semester 8/Large Language Models sem 8/Project/Code/BankAssist-chatbot/data/Dataset: funds transfer app features faqFile.json"
output_md_path = output_file
with open(input_json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

json_to_markdown(data, output_md_path, source_path=input_json_path)


In [ ]:
!pip install llama-index llama-index-llms-huggingface llama-index-readers-file > /dev/null 2>&1
!pip install -U  transformers accelerate > /dev/null 2>&1
!pip install -U bitsandbytes > /dev/null 2>&1

In [ ]:
!pip install python-dotenv > /dev/null 2>&1

In [ ]:
import nest_asyncio
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown

In [ ]:
nest_asyncio.apply()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
hf_token = os.getenv("HUGGINGFACE_TOKEN")
login(token=hf_token)


In [ ]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

In [ ]:
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
system_prompt = ("You are a helpful bank assistant. "
        "Answer user questions clearly and factually based only on the given context. "
        "Do not add greetings, sign-offs, or suggestions. "
        "Do not mention context or sources. Just answer the question directly."
)

In [ ]:
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    context_window=4096,
    max_new_tokens=512,
    system_prompt=system_prompt,
    generate_kwargs={"temperature": 0.3, "do_sample": False},
    model_kwargs={"torch_dtype": torch.float16}
)


In [ ]:
import re

def parse_markdown(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    pattern = re.compile(
        r'---\s*'                                     
        r'(?:sheet_name:\s*"(.*?)"\s*)?'              
        r'question:\s*"(.*?)"\s*'                     
        r'source:\s*"(.*?)"\s*---\s*'                 
        r'\*\*Answer:\*\*\s*(.*?)'                    
        r'(?=---|\Z)',                                
        re.DOTALL
    )

    qa_list = []
    for match in pattern.finditer(content):
        sheet_name, question, source, answer = match.groups()
        qa_list.append({
            'sheet_name': (sheet_name.strip() if sheet_name else None),
            'question': question.strip(),
            'source': source.strip(),
            'answer': answer.strip()
        })
    print('For verification purposes the QA list is as follows: \n', qa_list)
    return qa_list


In [ ]:
from llama_index.core import Document

def convert_to_documents(qa_list):
    documents = []
    for qa in qa_list:
        text = f"Q: {qa['question']}\nA: {qa['answer']}"
        metadata = {
            'sheet_name': qa['sheet_name'],
            'source': qa['source']
        }
        documents.append(Document(text=text, metadata=metadata))
    return documents


In [ ]:
documents = convert_to_documents(parse_markdown("bank_qna_md.md"))

In [ ]:
documents

In [ ]:
!pip install llama-index-embeddings-huggingface > /dev/null 2>&1

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5" 
)

print(f"Embedding model: {Settings.embed_model.model_name}")  

In [ ]:
Settings.llm = llm

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
index

In [ ]:

query_engine = index.as_query_engine(
    similarity_top_k=5,
    llm=llm
)

In [ ]:
query_1 = "What are the main features of NUST Sahar Account? Give brief overview over profit rate and payments. Answer in a paragrpah form"
retrieved_1 = query_engine.retrieve(query_1)
response_1 = query_engine.query(query_1)


In [ ]:
retrieved_1

In [ ]:
print("----------------------RESPONSE WITH LLAMA 3B----------------------")
display(Markdown(f"{response_1}"))


In [ ]:
query_2 = "What account options are available for women? Answer in a paragrpah form"
retrieved_2 = query_engine.retrieve(query_2)
response_2 = query_engine.query(query_2)


In [ ]:
retrieved_2

In [ ]:
print("----------------------RESPONSE WITH LLAMA 3B----------------------")
display(Markdown(f"{response_2}"))


In [ ]:
def append_excel_to_markdown(excel_file_path, markdown_path="bank_qna_md.md"):
    all_sheets = pd.read_excel(excel_file_path, sheet_name=None, header=None)
    appended = False

    with open(markdown_path, "a", encoding="utf-8") as f:
        for sheet_name, df in all_sheets.items():
            if sheet_name in ["Sheet3", "Sheet1", "Main"]:
                continue

            content = process_sheet(sheet_name, df, excel_file_path)
            if content:
                f.write("\n" + content + "\n")
                appended = True

    if appended:
        print(f"[INFO] New content from '{excel_file_path}' appended to '{markdown_path}'")
    else:
        print(f"[WARN] No new content found in '{excel_file_path}'")


In [ ]:
def refresh_rag_pipeline(markdown_path="bank_qna_md.md"):
    qa_list = parse_markdown(markdown_path)
    documents = convert_to_documents(qa_list)
    
    # Update embedding and LLM if needed
    Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    Settings.llm = llm
    
    index = VectorStoreIndex.from_documents(documents)
    query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)

    print("[INFO] RAG pipeline refreshed with updated data.")
    return query_engine


In [ ]:
!pip install ipywidgets > /dev/null 2>&1
!pip install openpyxl > /dev/null 2>&1 

import ipywidgets as widgets
from IPython.display import display

upload_widget = widgets.FileUpload(accept='.xlsx', multiple=False)
display(upload_widget)


In [ ]:
import os

def handle_uploaded_file(upload_widget):
    if not upload_widget.value:
        print("[ERROR] No file uploaded.")
        return None

    uploaded_file_info = upload_widget.value[0] 
    filename = uploaded_file_info['name']
    content = uploaded_file_info['content']

    file_path = os.path.join(os.getcwd(), filename)
    with open(file_path, "wb") as f:
        f.write(content)

    print(f"[INFO] Uploaded file saved to: {file_path}")
    return file_path

uploaded_file_path = handle_uploaded_file(upload_widget)

if uploaded_file_path:
    append_excel_to_markdown(uploaded_file_path, markdown_path="bank_qna_md.md")
    new_query_engine = refresh_rag_pipeline("bank_qna_md.md")

In [ ]:
new_query_1 = "What are the main features of NUST Sahar Account? Give brief overview over profit rate and payments. Answer in a paragrpah form"
new_response_1 = new_query_engine.query(new_query_1)

In [ ]:
print("----------------------RESPONSE WITH LLAMA 3B----------------------")
display(Markdown(f"{new_response_1}"))

In [ ]:
new_query_2 = "What account options are available for women? Answer in a paragrpah form"
new_response_2 = new_query_engine.query(new_query_2)

In [ ]:
print("----------------------RESPONSE WITH LLAMA 3B----------------------")
display(Markdown(f"{new_response_2}"))